In [1]:
⍝| echo: false
]box on -style=max
]rows on
⎕io←0

Rebuilding user command cache... done

┌→────────────────┐
│Was ON -style=min│
└─────────────────┘

┌→──────┐
│Was OFF│
└───────┘

# A New Kind of Science

A while back Stephen Wolfram published a massive tome called [A New Kind of Science](https://www.amazon.com/New-Kind-Science-Stephen-Wolfram-ebook/dp/B01N1I83V8). It can be pretty fun to reproduce some of the experiments. 

One thing he does is experiment with simple automata. These automata are repeated applications of very simple rules. 

Before we get to that. Becuase unicode is so easy to use with dyalog we can do some super cool stuff with it. For instance there are unicode characters for white and black squars □ ■. We can put these into an array and index into it. We can visualize binary arrays super easily.

In [13]:
⎕←colors←'□■'

┌→─┐
│□■│
└──┘

In [15]:
colors⌷⍨⊂1 0 1 1 0 0 1

┌→──────┐
│■□■■□□■│
└───────┘

Back to the automata the rules are super simple. You take a binary array. Chunk it up into windows. Each window will get mapped to a new bit either a 1 or a 0. For a window size of 3 You need 8 transformation rules. 

What does 0 0 0 map to?

What does 0 0 1 map to?

What does 0 1 0 map to?

etc. 

Each transformation can be to a 0 or 1 so in this universe there are a total of 256 rule sets. 

In [25]:
⍝ Example of rule 33 
(colors⌷⍨⊂⍉2 2 2⊤⍳8) (colors⌷⍨⊂⍪2 2 2 2 2 2 2 2⊤33)

┌→──────────┐
│ ┌→──┐ ┌→┐ │
│ ↓□□□│ ↓□│ │
│ │□□■│ │□│ │
│ │□■□│ │■│ │
│ │□■■│ │□│ │
│ │■□□│ │□│ │
│ │■□■│ │□│ │
│ │■■□│ │□│ │
│ │■■■│ │■│ │
│ └───┘ └─┘ │
└∊──────────┘

This is actually shockingly easy to implement. Lets first create a tool that generates are transformation rule

In [35]:
(8⍴2)∘⊤7

┌→──────────────┐
│0 0 0 0 0 1 1 1│
└~──────────────┘

This says 
000 -> 0
001 -> 0
010 -> 0
011 -> 0
100 -> 0
101 -> 1
110 -> 1
111 -> 1

In [36]:
rule←(8⍴2)∘⊤

In [26]:
⎕←p0←0 0 0 0 0 1 0 0 0 0 0

┌→────────────────────┐
│0 0 0 0 0 1 0 0 0 0 0│
└~────────────────────┘

In [28]:
⍝ Chunk p0 into windows of size 3
{⍵}⌺3⊢p0

┌→────┐
↓0 0 0│
│0 0 0│
│0 0 0│
│0 0 0│
│0 0 1│
│0 1 0│
│1 0 0│
│0 0 0│
│0 0 0│
│0 0 0│
│0 0 0│
└~────┘

To apply our rule we simply look up in our rule array the index that corresponds to the binary decoding of our window.

In [39]:
⍝ Binary decode the chunks
2⊥⍉{⍵}⌺3⊢p0

┌→────────────────────┐
│0 0 0 0 1 2 4 0 0 0 0│
└~────────────────────┘

In [41]:
⍝ look it up in rule 7
(rule 7)⌷⍨⊂2⊥⍉{⍵}⌺3⊢p0

┌→────────────────────┐
│0 0 0 0 0 0 0 0 0 0 0│
└~────────────────────┘

In [42]:
step←{(rule ⍺)⌷⍨⊂2⊥⍉{⍵}⌺3⊢⍵}

Lets wrap this up in a function so we can accumlate all the different stages

In [45]:
]dinput
run←{
    r c acc←⍺ ⍝ Takes a rule number, a count, and an accumulator.
    c=0:↑acc ⍝ if the count is 0 return the accumulator
    next←r step ⍵ ⍝ apply our step to the input
    r(c-1)(acc,⊂next)∇next ⍝recurse
}

In [49]:
⍝ run rule 54 for 10 steps on p0
(54 10, ⊂⊂p0) run p0

┌→────────────────────┐
↓0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
│0 0 0 0 0 1 0 0 0 0 0│
└~────────────────────┘

In [50]:
⍝ colorize it!
colors⌷⍨⊂(54 10, ⊂⊂p0) run p0

┌→──────────┐
↓□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
│□□□□□■□□□□□│
└───────────┘

Lets do a bigger example with a different rule

In [64]:
⎕←p101←50⌽101↑1

┌→────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
└~────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [65]:
colors⌷⍨⊂(74 50, ⊂⊂p101) run p101

┌→────────────────────────────────────────────────────────────────────────────────────────────────────┐
↓□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□□□□□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□□□□□□□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□□□□□□□□□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□■□■□□□□□■□■□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□■□■□□□□□□□□□□□□□□□□□□□□□■□■□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□■□□□■□□□□□□□□□□□□□□□□□□□■□□□■□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□■□■□■□■□□□□□□□□□□□□□□□□□■□■□■□■□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□□□□□□□□□■□■□□□□□□□□□□□□□■□■□□□□□□□□□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□□□□□□□□□■□□□■□□□□□□□□□□□■□□□■□□□□□□□□□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□□□□□□□□□■□■□■□■□□□□□□□□□■□■□■□■□□□□□□□□□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□■□■□□□□□■□■□□□□□■□■□□□□□■□■□□□□□■□■□□□□□■□■□□□□□■□■□□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□■□□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□■□□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□□■□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□■□□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□□■□■□■□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□■□■□■□□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□■□□□□□□□□□□□□□│
│□□□□□□□□□□□□□□■□■□□□□□■□■□□□□□□□□□□□□□□□□□□□□□□

## Lets look at every single rule

In [67]:
⎕←p51←25⌽51↑1

┌→────────────────────────────────────────────────────────────────────────────────────────────────────┐
│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
└~────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [68]:
16 16 ⍴ {colors⌷⍨⊂(⍵ 10, ⊂⊂p51) run p51}¨⍳256

┌→────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
↓ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ ┌→──────────────────────────────────────────────────┐ │
│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ ↓□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□□│ │
│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□□│ │
│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□□□□□│ │□□□□□□□□□□□□□□□□□□□□□□□□□□□□■□□□□□□□□□□